# Ceneo scraper

## Components of single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|Opinion ID|page_dom['data-entry-id']|opinion_id|
|opinion’s author|page_dom.select('span.user-post__author-name')|author|
|author’s recommendation|page_dom.select(".user-post__author-recomendation > em")|reccomendation|
|score expressed in number of stars|page_dom.select(".user-post__score-count")|score|
|opinion’s content|page_dom.select(".user-post__text")|content|
|list of product advantages|page_dom.select(".review-feature__title--positives ~ review-feature__item")|pros|
|list of product disadvantages|page_dom.select(".review-feature__title--negatives ~ review-feature__item"")|cons|
|how many users think that opinion was helpful|span[id^="votes-yes"]|helpful|
|how many users think that opinion was unhelpful|span[id^="votes-no"]|unhelpful|
|publishing date|('span.user-post__published > time:nth-child(1)['datetime']')|publish_date|
|purchase date|('span.user-post__published > time:nth-child(2)['datetime']')|purchase_date|

## Structure of single opinion

In [9]:
selectors = {
"opinion_id" :  [None , 'data-entry-id'],
"author" :  ['span.user-post__author-name'],
"recomendation" :  [".user-post__author-recomendation > em"],
"score" :  ["span.user-post__score-count"],
"content" :  [".user-post__text"],
"pros" :  ["div.review-feature__title--positives ~ div.review-feature__item", None , True,],
"cons" :  ["div.review-feature__title--negatives ~ div.review-feature__item", None , True,],
"helpful" :  ['button.vote-yes > span'],
"unhelpful" : [ 'button.vote-no > span'],
"publish" :  ['span.user-post__published > time:nth-child(1)' , 'datetime'],
"purchase" :  ["span.user-post__published > time:nth-child(2)" , 'datetime']
}

## loading lirbraries

In [10]:
import requests
from bs4 import BeautifulSoup

## Function to extract data from html code

In [15]:
def extract(ancestor, selector, attribute = None , return_list = False):
    if return_list:
        if attribute:
            return [p[attribute] for p in ancestor.select(selector)]
        return [p.get_text().strip() for p in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return  None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return None

## Sending request to Ceneo.pl server

In [12]:
product_id = '97571150;02517'
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)


## Extracting opinions from html code

In [13]:
page_dom = BeautifulSoup(response.text , 'html.parser')
opinions = page_dom.select('.js_product-review')
opinion = page_dom.select_one('.js_product-review')

## Extracting components from single opinion


In [16]:
single_opinion = {
    key : extract(opinion , *value)
        for key, value in selectors.items()
}